## Imports

In [1]:
from constants import FLUXES_FOLDER, SLM_FOLDER

from data_utils import process_amp_phase_data

from plot_utils import plot_map, \
                       plot_model_history, \
                       plot_fully_connected_amp_phase_prediction, \
                       plot_conv_amp_phase_prediction

from modeling_utils import create_fully_connected_architecture_for_amplitude_and_phase_reconstruction, \
                           create_convolutional_architecture_for_amplitude_and_phase_reconstruction, \
                           compile_model, \
                           train_model

from configurations import FirstConvolutionalModelWithBN as ModelConfiguration

2023-12-09 17:14:48.535544: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-09 17:14:48.809105: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-09 17:14:48.809218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-09 17:14:48.842140: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-09 17:14:48.936637: I tensorflow/core/platform/cpu_feature_guar

## Load data

In the first place, define the data paths

In [2]:
train_fluxes_array, val_fluxes_array, train_amp_phase_array, val_amp_phase_array, scalers = \
    process_amp_phase_data(
        n_points=40000,
        normalize_flux=True,
        normalize_amplitude=True,
        shuffle=False,
        flatten_fluxes=False,
        split=True,
        val_ratio=0.1
    )

In [3]:
train_fluxes_array.shape

(36000, 55, 24)

In [4]:
val_fluxes_array.shape

(4000, 55, 24)

In [5]:
train_amp_phase_array.shape

(36000, 2, 96, 96)

In [6]:
val_amp_phase_array.shape

array([[[[ 6.98578053e-01,  7.43319937e-01,  7.90347719e-01, ...,
           4.21478524e-01,  5.03408261e-01,  5.77518393e-01],
         [ 5.44036523e-01,  5.83540890e-01,  6.25700029e-01, ...,
           3.14985327e-01,  3.98666633e-01,  4.74617017e-01],
         [ 3.80387767e-01,  4.14716237e-01,  4.52099906e-01, ...,
           2.11574432e-01,  2.96703474e-01,  3.74210814e-01],
         ...,
         [ 3.47608817e-01,  2.76880737e-01,  1.98129718e-01, ...,
           3.06610904e-01,  2.70855147e-01,  2.38132616e-01],
         [ 4.43536922e-01,  3.74210309e-01,  2.96703970e-01, ...,
           4.92059931e-01,  4.52099391e-01,  4.14716747e-01],
         [ 5.42260586e-01,  4.74616499e-01,  3.98667141e-01, ...,
           6.70015776e-01,  6.25699493e-01,  5.83541421e-01]],

        [[-9.20236727e-02, -5.71971023e-02, -2.34401611e-02, ...,
           5.14060277e-01,  5.31800416e-01,  5.47601037e-01],
         [-9.58859807e-02, -5.92038460e-02, -2.36001412e-02, ...,
           5.41211293e

## CONVOLUTIONAL MODEL

In [7]:
model_configuration = ModelConfiguration(train_fluxes_array,
                                         train_amp_phase_array)
print(model_configuration.get_description())


	=== ConvolutionalAmplitudePhaseReconstructor ===
	*ARCHITECTURE HYPERPARAMETERS:
		-Convolutional
		-Input shape: (55, 24)
		-Output shape: (2, 96, 96)
		-Convolutional Layers: [128, 64]
		-Convolutonal Kernels: [(5, 5), (3, 3)]
		-Fully Connected Hidden layers: [1024, 2048, 2048, 2048]
		-Regularizer: None
		-Convolutional Activation: relu
		-Hidden Layers Activation: relu
		-Output Layer Activation: linear
		-Batch Normalization: True
	
	*COMPILATION HYPERPARAMETERS:
		-Optimizer: ADAM lr=0.0001, beta_1=0.9, beta_2=0.999
		-Loss Function: MSE
		-Metric: MSE
	
	* TRAINING HYPERPARAMETERS:
		-Epochs: 1000
		-Batch size: 32
		-Callbacks:
			-ReduceLROnPlateau: MSE 15 x0.1
			-Early Stop: MSE 50
	


2023-12-09 17:15:21.988851: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-09 17:15:22.092527: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-09 17:15:22.093056: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [8]:
model = create_convolutional_architecture_for_amplitude_and_phase_reconstruction(
    *model_configuration.unpack_architecture_hyperparameters()
)
compile_model(
    model,
    *model_configuration.unpack_compilation_hyperparameters()
)

Model: "ConvolutionalAmplitudePhaseReconstructor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 51, 20, 128)       3328      
                                                                 
 max_pooling2d (MaxPooling2  (None, 25, 10, 128)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 8, 64)         73792     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 11, 4, 64)         0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 2816)              0         
                                                                 
 dense (Dense)            

In [ ]:
history = train_model(
    model,
    train_fluxes_array,
    train_amp_phase_array,
    val_fluxes_array,
    val_amp_phase_array,
    *model_configuration.unpack_training_hyperparameters()
)

Epoch 1/1000


2023-12-09 17:15:27.024630: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-09 17:15:27.162873: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-09 17:15:27.766410: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-09 17:15:28.142790: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fe28e998d20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-09 17:15:28.142805: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2023-12-09 17:15:28.158724: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1702102528.326905   51083 devic

1125/1125 [==============================] - 27s 20ms/step - loss: 0.6155 - mean_squared_error: 0.6155 - val_loss: 1.2712 - val_mean_squared_error: 1.2712 - lr: 1.0000e-04
Epoch 2/1000
1125/1125 [==============================] - 22s 19ms/step - loss: 0.5450 - mean_squared_error: 0.5450 - val_loss: 1.3449 - val_mean_squared_error: 1.3449 - lr: 1.0000e-04
Epoch 3/1000
1125/1125 [==============================] - 22s 19ms/step - loss: 0.5136 - mean_squared_error: 0.5136 - val_loss: 1.4042 - val_mean_squared_error: 1.4042 - lr: 1.0000e-04
Epoch 4/1000
1125/1125 [==============================] - 22s 20ms/step - loss: 0.4909 - mean_squared_error: 0.4909 - val_loss: 1.4790 - val_mean_squared_error: 1.4790 - lr: 1.0000e-04
Epoch 5/1000
1125/1125 [==============================] - 22s 20ms/step - loss: 0.4750 - mean_squared_error: 0.4750 - val_loss: 1.1995 - val_mean_squared_error: 1.1995 - lr: 1.0000e-04
Epoch 6/1000
1125/1125 [==============================] - 22s 20ms/step - loss: 0.4600 -

In [ ]:
plot_model_history(history)

In [ ]:
n = 106
plot_conv_amp_phase_prediction(
    model,
    val_fluxes_array[n],
    val_amp_phase_array[n][0],
    val_amp_phase_array[n][1]
    )

In [ ]:
n = 106
plot_conv_amp_phase_prediction(
    model,
    train_fluxes_array[n],
    train_amp_phase_array[n][0],
    train_amp_phase_array[n][1]
    )